In [0]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

The Dataset is taken form :https://www.kaggle.com/deadskull7/fer2013


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, Dropout, MaxPooling2D, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import optimizers
from sklearn.metrics import accuracy_score
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from keras.utils import to_categorical


Using TensorFlow backend.


In [0]:
def load_data(data_path):
    train_data=[]
    train_label=[]
    test_data=[]
    test_label=[]
    with open(data_path,'r') as file:
        for line_no,line in enumerate(file.readlines()):
            if(0<line_no<=35887):
                class_type,pixels,use_type=line.split(',')
                
                #In dataest images are in 48*48 dimesion.So we extract the image data and rearrange it to 48*48 array
                image_data=np.asarray([int(x) for x in pixels.split()]).reshape(48,48)  
                
                #We convert the image data into unsigned integer and divide it by 255 to normalize the data.
                #255 is the maximum possible value of a single cell. By dividing every element by 255, we ensure that all our values range between 0 and 1.
                image_data=image_data.astype(np.uint8)/255.0
              
                if (use_type.strip() == 'PrivateTest'):    
                    test_data.append(image_data)
                    test_label.append(class_type)
                else:
                    train_data.append(image_data)
                    train_label.append(class_type)
        
        #we will expand the dimensions of both testing and training data by one to accommodate the channel 
        test_data=np.expand_dims(test_data,-1)
        train_data=np.expand_dims(train_data,-1)
        
        #one hot encode all the labels using the to_categorical() function 
        test_label=to_categorical(test_label, num_classes=7)
        train_label=to_categorical(train_label, num_classes=7)
        
        return np.array(train_data),np.array(train_label),np.array(test_data),np.array(test_label)

In [0]:
#load the data
path="/content/gdrive/My Drive/Colab Notebooks/fer2013.csv"
train_data,train_label,test_data,test_label=load_data(path)
print("Number of images in Training set:", len(train_data))
print("Number of images in Test set:", len(test_data))

Number of images in Training set: 32298
Number of images in Test set: 3589


In [0]:
model=Sequential()

model.add(Conv2D(64,(3,3),input_shape=(48,48,1),activation='relu',kernel_regularizer=l2(0.01)))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7,activation='softmax'))

epochs = 100
batch_size = 64
learning_rate = 0.001

adam=optimizers.Adam(lr=learning_rate)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary)

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
#early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('/content/gdrive/My Drive/Colab Notebooks/weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_label,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.2,
          shuffle = True,
          callbacks=[lr_reducer,checkpointer]
          )

W0625 22:41:59.267386 139710620465024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0625 22:41:59.283449 139710620465024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 22:41:59.286869 139710620465024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0625 22:41:59.326290 139710620465024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0625 22:41:59.327161 1397106204

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7f1098794748>>
Train on 25838 samples, validate on 6460 samples
Epoch 1/100
25838/25838 [==============================] - 38s 1ms/step - loss: 2.0823 - acc: 0.2052 - val_loss: 1.8720 - val_acc: 0.2489

Epoch 00001: val_loss improved from inf to 1.87198, saving model to /content/gdrive/My Drive/Colab Notebooks/weights.hd5
Epoch 2/100
25838/25838 [==============================] - 32s 1ms/step - loss: 1.8663 - acc: 0.2414 - val_loss: 1.8432 - val_acc: 0.2489

Epoch 00002: val_loss improved from 1.87198 to 1.84319, saving model to /content/gdrive/My Drive/Colab Notebooks/weights.hd5
Epoch 3/100
25838/25838 [==============================] - 32s 1ms/step - loss: 1.8428 - acc: 0.2482 - val_loss: 1.8261 - val_acc: 0.2489

Epoch 00003: val_loss improved from 1.84319 to 1.82614, saving model to /content/gdrive/My Drive/Colab Notebooks/weights.hd5
Epoch 4/100
25838/25838 [==============================] - 32s 1ms/

In [0]:
predicted_test_label = np.argmax(model.predict(test_data), axis=1)
test_label = np.argmax(test_label, axis=1)
print ("Accuracy score = ", accuracy_score(test_label, predicted_test_label))

Accuracy score =  0.5611590972415714
